In [1]:
import math
from decimal import Decimal, ROUND_HALF_UP

from scipy.special import comb
from scipy.stats import poisson

In [2]:
def combt(n, r):
    return comb(n, r, exact=True)

## 概率计算

In [3]:
def total_cases():
    return combt(33, 6) * 16

def level_1_cases():
    """一等奖：投注号码与当期开奖号码全部相同（顺序不限，下同），即中奖；"""
    return 1

def level_2_cases():
    """二等奖：投注号码与当期开奖号码中的6个红色球号码相同，即中奖；"""
    return 16 - 1

def level_3_cases():
    """三等奖：投注号码与当期开奖号码中的任意5个红色球号码和1个蓝色球号码相同，即中奖；"""
    return combt(6, 5) * (33 - 6)

def level_4_cases():
    """四等奖：投注号码与当期开奖号码中的任意5个红色球号码相同，或与任意4个红色球号码和1个蓝色球号码相同，即中奖；"""
    return combt(6, 5) * (33 - 6) * (16 - 1) + combt(6, 4) * combt(33 - 6, 2)

def level_5_cases():
    """五等奖：投注号码与当期开奖号码中的任意4个红色球号码相同，或与任意3个红色球号码和1个蓝色球号码相同，即中奖；"""
    return combt(6, 4) * combt(33 - 6, 2) * (16 - 1) + combt(6, 3) * combt(33 - 6, 3)

def level_6_cases():
    """六等奖：投注号码与当期开奖号码中的1个蓝色球号码相同，即中奖。"""
    return combt(6, 2) * combt(33 - 6, 4) + combt(6, 1) * combt(33 - 6, 5) + combt(33 - 6, 6)

In [4]:
def diff(sales, one, two, three, four, five, six):
    """Check how the real data differentiate from the expected data"""
    volume = sales / 2
    
    cases = [level_1_cases(), level_2_cases(), level_3_cases(), level_4_cases(), level_5_cases(), level_6_cases()]
    real = [one, two, three, four, five, six]
    
    probabilities = [case / total_cases() for case in cases]
    expected = [volume * p for p in probabilities]
    std = [math.sqrt(volume * p * (1 - p)) for p in probabilities]
    
    diff_ratio = [real[i] / expected[i] - 1 for i in range(6)]
    std_ratio = [std[i] / expected[i] for i in range(6)]
    
    return diff_ratio, std_ratio

### 验证中奖数是否与理论值接近

In [5]:
# 2020032: 03, 11, 13, 14, 15, 26; 13
diff(338_904_002, 11, 135, 1606, 79333, 1458725, 8679754)

([0.15036686996691184,
  -0.05879074275434493,
  0.03675038898252536,
  0.07817419688209593,
  0.10966880748068797,
  -0.1302380966387937],
 [0.3233864568272026,
  0.08349799112952193,
  0.025407524201210945,
  0.0036857274424272027,
  0.000868798061792571,
  0.0003070902929542848])

In [6]:
# 2020031: 02, 05, 09, 15, 16, 27; 09
diff(323_903_392, 14, 310, 3568, 140308, 1985178, 11566940)

([0.5319088229863305,
  1.2613892148845829,
  1.409987072493486,
  0.9951644215684214,
  0.5800855705704191,
  0.21275381227198675],
 [0.33079004309388943,
  0.08540958812862814,
  0.025989202231553175,
  0.0037701082212119054,
  0.0008886882620870397,
  0.0003141207960181903])

In [7]:
# 2020030: 17, 18, 21, 29, 30, 32; 03
diff(327_786_632, 1, 74, 894, 44840, 944906, 8805838)

([-0.8918742482457308,
  -0.46657962467893854,
  -0.40330603661532916,
  -0.36993389101215957,
  -0.2568199920922385,
  -0.08767596721445081],
 [0.32882479476575227,
  0.08490216339266343,
  0.02583479844189412,
  0.0037477097269605843,
  0.0008834084988119222,
  0.00031225458093071665])

In [8]:
# 2020029: 01 12 18 20 30 32 05
diff(364_268_814, 7, 63, 983, 56195, 1103123, 13626479)

([-0.3189226679174353,
  -0.5913536007504612,
  -0.4096128594028562,
  -0.28946179009784234,
  -0.21927408121069814,
  0.270373194232151],
 [0.31192427965073494,
  0.0805384708775124,
  0.024506974602235988,
  0.003555089748493583,
  0.0008380042016769036,
  0.0002962057204166779])

In [9]:
# 2020028: 05 06 15 18 26 32 08
diff(323_795_596, 7, 153, 1247, 77287, 1398236, 7837568)

([-0.2337905917658003,
  0.11647656628411962,
  -0.15743991881124608,
  0.09937856742219608,
  0.1132845744751041,
  -0.17798442548945437],
 [0.3308451007856451,
  0.0854238039578961,
  0.02599352795270263,
  0.0037707357293871443,
  0.0008888361780398394,
  0.0003141730792302225])

### 实际开出的中奖数往往偏离理论的期望值很多，可能是由于彩民选数字不是完全随机，而是有一定的偏好。

## 奖池计算

In [10]:
def pool_diff(sales, one, two, three, four, five, six, previous_pool, next_pool, level_1_bonus, level_2_bonus):
    """Check if the level 1 and 2 bonus and reward pool is calculated correctly"""
    
    sales = Decimal(sales)
    one = Decimal(one)
    two = Decimal(two)
    three = Decimal(three)
    four = Decimal(four)
    five = Decimal(five)
    six = Decimal(six)
    previous_pool = Decimal(previous_pool)
    next_pool = Decimal(next_pool)
    level_1_bonus = Decimal(level_1_bonus)
    level_2_bonus = Decimal(level_2_bonus)

    lower_level_reward = three * Decimal(3000) + four * Decimal(200) + five * Decimal(10) + six * Decimal(5)
    current_reward = sales * Decimal("0.49")
    current_high_level_reward = current_reward - lower_level_reward
    
    expect_level_2_bonus = (current_high_level_reward * Decimal("0.25")) / two
    expect_level_2_bonus = expect_level_2_bonus.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
    expect_level_2_bonus = min(expect_level_2_bonus, Decimal(5_000_000))
    
    if one == 0:
        expect_level_1_bonus = Decimal(0)
        level_1_reward = Decimal(0)
    else:
        if previous_pool < 100_000_000:
            expect_level_1_bonus = (current_high_level_reward * Decimal("0.75") + previous_pool) / one
            expect_level_1_bonus = expect_level_1_bonus.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
            expect_level_1_bonus = min(expect_level_1_bonus, Decimal(5_000_000))
        else:
            part_1 = (current_high_level_reward * Decimal("0.55") + previous_pool) / one
            part_1 = part_1.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
            part_1 = min(part_1, Decimal(5_000_000))

            part_2 = current_high_level_reward * Decimal("0.2") / one
            part_2 = part_2.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
            part_2 = min(part_2, Decimal(5_000_000))

            expect_level_1_bonus = part_1 + part_2
        
        level_1_reward = one * expect_level_1_bonus
        
    expected_next_pool = previous_pool + current_reward - lower_level_reward \
        - level_1_reward - two * expect_level_2_bonus
    
    return (
        math.floor(expect_level_1_bonus) - level_1_bonus,
        math.floor(expect_level_2_bonus) - level_2_bonus,
        expected_next_pool - next_pool
    )

In [11]:
# 2020032
pool_diff(338_904_002, 11, 135, 1606, 79333, 1458725, 8679754, 850135149, 843200936, 6588951, 161837)

(Decimal('0'), Decimal('0'), Decimal('0.38'))

In [12]:
# 2020031
pool_diff(323_903_392, 14, 310, 3568, 140308, 1985178, 11566940, 896891829, 850135149, 5603722, 34081)

(Decimal('0'), Decimal('0'), Decimal('1.58'))

In [13]:
# 2020030
pool_diff(327_786_632, 1, 74, 894, 44840, 944906, 8805838, 835276431, 896891829, 10000000, 322591)

(Decimal('0'), Decimal('0'), Decimal('1.82'))

In [14]:
# 2020029
pool_diff(364_268_814, 7, 63, 983, 56195, 1103123, 13626479, 823449380, 835276431, 7432574, 337857)

(Decimal('0'), Decimal('0'), Decimal('0.33'))

In [15]:
# 2020028
pool_diff(323_795_596, 7, 153, 1247, 77287, 1398236, 7837568, 810989197, 823449380, 7465464, 140998)

(Decimal('0'), Decimal('0'), Decimal('0.34'))

In [16]:
# 2017029
pool_diff(343_011_048, 4, 53, 1118, 57929, 1106505, 11725496, 977766651, 1003660344, 9172154, 393599)

(Decimal('0'), Decimal('0'), Decimal('2.95'))

In [17]:
# 2017028
pool_diff(377_657_484, 9, 152, 2120, 100177, 1661969, 13106644, 980689536, 977766651, 6700085, 125828)

(Decimal('0'), Decimal('0'), Decimal('6.85'))

In [18]:
# 2018094 no level 1
pool_diff(314_204_036, 0, 148, 1301, 62638, 1144407, 8907745, 962368068, 1023528004, 0, 137747)

(Decimal('0'), Decimal('0'), Decimal('0.36'))

### 计算结果有些许误差，不太清楚小数部分是如何计算的。

## 获奖期望

In [19]:
def low_level_expectation():
    return (level_3_cases() * 3000 + level_4_cases() * 200 + level_5_cases() * 10 + level_6_cases() * 5) / total_cases()

In [20]:
def total_expectation(pool, sales, multiplier):
    """Expected reward of one ticket"""
    # Here we ignore the randomness of the numbers, the result can be inaccurate
    other_people_level_1 = sales / 2 * level_1_cases() / total_cases()
    other_people_level_2 = sales / 2 * level_2_cases() / total_cases()
    
    lower_level_reward = sales / 2 * low_level_expectation()
    current_reward = sales * 0.49
    current_high_level_reward = current_reward - lower_level_reward
    
    expect_level_2_bonus = (current_high_level_reward * 0.25) / (other_people_level_2 + multiplier)
    print(f"Level 2 reward: {expect_level_2_bonus}")
    expect_level_2_reward = expect_level_2_bonus * level_2_cases() / total_cases()
    
    if pool < 100_000_000:
        expect_level_1_bonus = (current_high_level_reward * 0.75 + pool) / (other_people_level_1 + multiplier)
        print(f"Level 1 reward: {expect_level_1_bonus}")
        expect_level_1_bonus = min(expect_level_1_bonus, 5_000_000)
    else:
        part_1 = (current_high_level_reward * 0.55 + pool) / (other_people_level_1 + multiplier)
        print(f"Level 1 reward part 1 {part_1}")
        part_1 = min(part_1, 5_000_000)
        
        part_2 = current_high_level_reward * 0.2 / (other_people_level_1 + multiplier)
        print(f"Level 1 reward part 2 {part_2}")
        part_2 = min(part_2, 5_000_000)
        
        expect_level_1_bonus = part_1 + part_2
        
    expect_level_1_reward = expect_level_1_bonus * level_1_cases() / total_cases()
    
    return expect_level_1_reward + expect_level_2_reward + low_level_expectation()

In [21]:
total_expectation(80_000_000, 338_000_000, 1)

Level 2 reward: 144799.70525475597
Level 1 reward: 13531341.588048158


0.8910257416938137

In [22]:
total_expectation(99_999_999, 1e14, 1)

Level 2 reward: 145811.93388807186
Level 1 reward: 6561570.296612353


0.891882541767248

In [23]:
total_expectation(1e17, 1e15, 1)

Level 2 reward: 145811.93698880717
Level 1 reward part 1 3549029268.14668
Level 1 reward part 2 1749743.185985294


0.990620454049853

In [24]:
def upper_limit_less_than_hundred_million():
    level_2_limit = (0.98 - low_level_expectation()) * 0.25
    level_1_limit = 5_000_000 * level_1_cases() / total_cases()
    return low_level_expectation() + level_2_limit + level_1_limit

In [25]:
upper_limit_less_than_hundred_million()

0.8918825446834868

In [26]:
def upper_limit_greater_than_hundred_million():
    level_2_limit = (0.98 - low_level_expectation()) * 0.25
    level_1_part_1_limit = 5_000_000 * level_1_cases() / total_cases()
    level_1_part_2_limit = min((0.98 - low_level_expectation()) * 0.2, 5_000_000 * level_1_cases() / total_cases())
    return low_level_expectation() + level_2_limit + level_1_part_1_limit + level_1_part_2_limit

In [27]:
upper_limit_greater_than_hundred_million()

0.9906204578409634

### 可见无论奖池里的金额有多大，一张彩票收益的期望都小于0.9907元

## 奖池增长

In [28]:
def pool_growth(previous_pool, sales, one):
    lower_level_reward = sales / 2 * low_level_expectation()
    current_reward = sales * 0.49
    current_high_level_reward = current_reward - lower_level_reward

    level_2_reward = current_high_level_reward * 0.25
    
    if one == 0:
        level_1_reward = 0
    else:
        if previous_pool < 100_000_000:
            expect_level_1_bonus = (current_high_level_reward * 0.75 + previous_pool) / one
            expect_level_1_bonus = min(expect_level_1_bonus, 5_000_000)
        else:
            part_1 = (current_high_level_reward * 0.55 + previous_pool) / one
            part_1 = min(part_1, 5_000_000)

            part_2 = current_high_level_reward * 0.2 / one
            part_2 = min(part_2, 5_000_000)

            expect_level_1_bonus = part_1 + part_2
        
        level_1_reward = one * expect_level_1_bonus
    
    return current_reward - lower_level_reward - level_1_reward - level_2_reward

In [29]:
def expect_pool_growth(previous_pool, sales):
    one = 0
    acc = 0.0
    while True:
        growth = pool_growth(previous_pool, sales, one)
        left = previous_pool + growth
        
        if left / sales < 1e-8:
            break
            
        acc += left * poisson.pmf(one, mu=sales / 2 * level_1_cases() / total_cases())
        one += 1
    
    return acc - previous_pool

In [30]:
pool_growth(0.9e8, 3e8, 8.464491570720714) / (3e8 / 2)

0.0881174553165133

In [31]:
0.98 - upper_limit_less_than_hundred_million()

0.08811745531651316

In [32]:
expect_pool_growth(0.9e8, 3e8) / (3e8 / 2)

0.08811745562339991

In [33]:
pool_growth(10e8, 3e8, 8.464491570720714) / (3e8 / 2)

-0.01062045784096323

In [34]:
0.98 - upper_limit_greater_than_hundred_million()

-0.010620457840963438

In [35]:
expect_pool_growth(10e8, 3e8) / (3e8 / 2)

-0.010240708958779175

### 计算结果与上面基本一致：奖池少于一亿时奖池会增长，奖池大于一亿时会微微减少